In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import numpy as np
import cv2
import tifffile as tiff
from pathlib import Path
from tqdm import tqdm

# =====================================
# USER SETTINGS
# =====================================

# Three INPUT folders
input_dirs = [
    "/content/drive/MyDrive/soil_microCT_images/ROI/bnei_reem_ROI_8bit",
    "/content/drive/MyDrive/soil_microCT_images/ROI/mishmar_hanegev_maoz_ROI_8bit",
    "/content/drive/MyDrive/soil_microCT_images/ROI/rehovot_ROI_8bit",
]

# Root folder where OUTPUT folders will be created
output_base = "/content/drive/MyDrive/soil_microCT_images/ROI/OUTPUT_MEDIAN_K3"

# Fixed median kernel size
median_k = 3
# =====================================

output_base = Path(output_base)
output_base.mkdir(parents=True, exist_ok=True)

print("Input → Output mapping:")

# =====================================
# PROCESS
# =====================================

for in_dir in input_dirs:
    in_dir = Path(in_dir)

    if not in_dir.exists():
        print(f"❌ Input folder not found: {in_dir}")
        continue

    # Create a SEPARATE output folder with the SAME name
    out_dir = output_base / in_dir.name
    out_dir.mkdir(parents=True, exist_ok=True)

    print(f" - {in_dir.name} → {out_dir}")

    files = sorted(in_dir.glob("*.tif"))

    for f in tqdm(files, desc=in_dir.name):
        # Read TIFF image
        img = tiff.imread(str(f))

        # If multi-channel TIFF, keep first channel
        if img.ndim == 3:
            img = img[..., 0]

        # Ensure uint8 for OpenCV median filter
        if img.dtype != np.uint8:
            img = img.astype(np.float32)
            lo, hi = np.percentile(img, 1), np.percentile(img, 99)
            img = np.clip(img, lo, hi)
            img = ((img - lo) / (hi - lo) * 255).astype(np.uint8)

        # Apply median filter
        filtered = cv2.medianBlur(img, median_k)

        # Save to corresponding output folder
        out_path = out_dir / f.name
        tiff.imwrite(str(out_path), filtered)

print("\n DONE: Each input folder was saved into its own output folder")


Input → Output mapping:
 - bnei_reem_ROI_8bit → /content/drive/MyDrive/soil_microCT_images/ROI/OUTPUT_MEDIAN_K3/bnei_reem_ROI_8bit


bnei_reem_ROI_8bit: 100%|██████████| 652/652 [04:44<00:00,  2.29it/s]


 - mishmar_hanegev_maoz_ROI_8bit → /content/drive/MyDrive/soil_microCT_images/ROI/OUTPUT_MEDIAN_K3/mishmar_hanegev_maoz_ROI_8bit


mishmar_hanegev_maoz_ROI_8bit: 100%|██████████| 1203/1203 [08:46<00:00,  2.29it/s]


 - rehovot_ROI_8bit → /content/drive/MyDrive/soil_microCT_images/ROI/OUTPUT_MEDIAN_K3/rehovot_ROI_8bit


rehovot_ROI_8bit:  32%|███▏      | 212/661 [01:25<03:01,  2.47it/s]


KeyboardInterrupt: 